In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch

import pickle

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

/home/ubuntu/anaconda3/envs/bnntest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [4]:
device = torch.device("cuda")

In [5]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

class BayesianCNNSingleFC(PyroModule):
    def __init__(
        self,
        num_classes,
        device,
        activation='sigmoid',
        prior_dist='gaussian',
        mu=0,
        b=10.0,
        prior_params=None
    ):
        super().__init__()

        # Store device
        self.device = device

        # Activation setup: accept string or callable
        if isinstance(activation, str):
            act_map = {
                'relu': F.relu,
                'tanh': F.tanh,
                'wg': self.actWG,
                'rwg': self.actRWG,
                'sigmoid': F.sigmoid,
                'sinusoidal': torch.sin,
                'relu6': F.relu6,
                'leaky_relu': F.leaky_relu,
                'selu': F.selu,
            }
            try:
                self.activation_fn = act_map[activation]
            except KeyError:
                raise ValueError(f"Unsupported activation: {activation}")
        elif callable(activation):
            self.activation_fn = activation
        else:
            raise ValueError("activation must be a string or callable")

        # Prior distribution setup
        self.prior_dist = prior_dist
        # Set default prior parameters if not provided
        default_params = {'mu': 0.0, 'b': 10.0}
        params = default_params if prior_params is None else prior_params
        self.prior_mu = torch.tensor(params.get('mu', params['mu']), device=device)
        self.prior_b  = torch.tensor(params.get('b', params['b']), device=device)

        print(f"Using prior distribution: {self.prior_dist} with mu={self.prior_mu.item()} and b={self.prior_b.item()}")

        # Layer definitions with priors
        self.conv1 = PyroModule[nn.Conv2d](3, 32, kernel_size=5, stride=1, padding=2)
        self.conv1.weight = PyroSample(self._make_prior([32, 3, 5, 5]))
        self.conv1.bias   = PyroSample(self._make_prior([32]))
        self.bn1 = nn.BatchNorm2d(32)  # Use nn.BatchNorm2d directly, no PyroModule needed

        self.conv2 = PyroModule[nn.Conv2d](32, 64, kernel_size=5, stride=1, padding=2)
        self.conv2.weight = PyroSample(self._make_prior([64, 32, 5, 5]))
        self.conv2.bias   = PyroSample(self._make_prior([64]))
        self.bn2 =  nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap  = nn.AdaptiveAvgPool2d((1, 1))

        self.fc1 = PyroModule[nn.Linear](64, num_classes)
        self.fc1.weight = PyroSample(self._make_prior([num_classes, 64]))
        self.fc1.bias   = PyroSample(self._make_prior([num_classes]))

    def actWG(self, x, alpha=1.0):
        # Weight-gradient activation
        return x * torch.exp(-alpha * x ** 2)
    
    def actRWG(self, x, alpha=1.0):
        wg = x * torch.exp(-alpha * x ** 2)
        # compare elementwise with zero
        return torch.max(torch.zeros_like(wg), wg)

    def _make_prior(self, shape):
        """
        Construct a prior distribution based on self.prior_dist and parameters.
        """
        if self.prior_dist == 'gaussian':
            base = dist.Normal(self.prior_mu, self.prior_b)
        elif self.prior_dist == 'laplace':
            base = dist.Laplace(self.prior_mu, self.prior_b)
        elif self.prior_dist == 'uniform':
            # uniform over [-b, b]
            base = dist.Uniform(-self.prior_b, self.prior_b)
        else:
            raise ValueError(f"Unsupported prior distribution: {self.prior_dist}")
        return base.expand(shape).to_event(len(shape))

    def forward(self, x, y=None):
        # x: [B, 3, 64, 64]
        #x = self.activation_fn(self.conv1(x).to(self.device))
        x = self.conv1(x).to(self.device)
        x = self.bn1(x)
        x = self.activation_fn(x)
        x = self.pool(x)
        #x = self.activation_fn(self.conv2(x))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation_fn(x)
        x = self.pool(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        logits = self.fc1(x)

        if y is not None:
            with pyro.plate("data", x.size(0)):
                pyro.sample("obs", dist.Categorical(logits=logits), obs=y)

        return logits

In [6]:
eurosat_mean = [0.344, 0.380, 0.408]
eurosat_std  = [0.190, 0.137, 0.115]

old_mean = [0.3444, 0.3803, 0.4078]
old_std = [0.0914, 0.0651, 0.0552]


def load_data(batch_size=54):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=eurosat_mean, 
                             std=eurosat_std)
    ])

    dataset = datasets.EuroSAT(root='./data', transform=transform, download=True)

    torch.manual_seed(42)

    #train_size = int(0.8 * len(dataset))
    #test_size = len(dataset) - train_size
    #train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    with open('datasplit/split_indices.pkl', 'rb') as f:
        split = pickle.load(f)
        train_dataset = Subset(dataset, split['train'])
        test_dataset = Subset(dataset, split['test'])

    # Add num_workers and pin_memory for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

In [7]:
import torch
import pyro.poutine as poutine
from pyro.infer import Trace_ELBO

elbo = Trace_ELBO()


import torch
import pyro.poutine as poutine

def compute_kl_nll_full(model, guide, x, y):
    # 1) Trace the guide (latent-only)
    guide_trace = poutine.trace(guide).get_trace(x, y)
    # 2) Replay the model with those latent samples
    model_trace = poutine.trace(
        poutine.replay(model, trace=guide_trace)
    ).get_trace(x, y)

    kl  = torch.tensor(0., device=x.device)
    nll = torch.tensor(0., device=x.device)

    def sum_lp(site):
        # Try built-in sums first
        if "log_prob_sum" in site:
            return site["log_prob_sum"]
        if "log_prob" in site:
            return site["log_prob"].sum()
        # Fallback to calling fn.log_prob
        lp = site["fn"].log_prob(site["value"])
        return lp.sum()

    # KL from all latent sites in the guide
    for name, site in guide_trace.nodes.items():
        if site["type"] != "sample" or site["is_observed"]:
            continue
        if name not in model_trace.nodes:
            continue
        q_lp = sum_lp(site)
        p_lp = sum_lp(model_trace.nodes[name])
        kl   = kl + (q_lp - p_lp)

    # NLL from the observed "obs" site in the model
    for name, site in model_trace.nodes.items():
        if site["type"] == "sample" and site["is_observed"]:
            # negative log-likelihood = -sum log p(y|x,w)
            nll = nll - sum_lp(site)

    return kl, nll


In [8]:
# training SVI function

import os
import torch
import pyro
from tqdm import tqdm
import numpy as np

def train_svi_with_stats(
    model,
    guide,
    svi,
    train_loader,
    device,
    num_epochs=10,
    save_epochs=None,
    save_dir='results',
    model_filename_pattern='model_{activation}_{prior}_epoch_{epoch}_{timestamp}.pth',
    guide_filename_pattern='guide_{activation}_{prior}_epoch_{epoch}_{timestamp}.pth',
    param_store_filename_pattern='param_store_{activation}_{prior}_epoch_{epoch}_{timestamp}.pkl',
    accuracies_filename_pattern='accuracy_results_{activation}_{prior}_{timestamp}.csv',
    losses_filename_pattern='losses_{activation}_{prior}_{timestamp}.csv',
    model_config_filename_pattern='config_{activation}_{prior}_{timestamp}.json'
):
    """
    Train the SVI model, track losses/accuracies, and
    save artifacts only when accuracy improves, naming files
    like `model_relu_gaussian_epoch_3.pth`.
    """
    
    # Pull names off the model if available, else fall back
    #act_name  = getattr(model, 'activation', getattr(model, 'activation_name', 'act'))
    act_name = model.activation_fn.__name__ if hasattr(model.activation_fn, '__name__') else str(model.activation_fn)
    prior_name = getattr(model, 'prior_dist', 'prior')
    timestamp = time.strftime("%Y%m%d_%H%M%S")

    os.makedirs(save_dir, exist_ok=True)
    save_epochs = set(save_epochs or range(1, num_epochs+1))

    pyro.clear_param_store()
    model.to(device)

    epoch_losses, epoch_accuracies, accuracy_epochs = [], [], []
    weight_stats = {'epochs': [], 'means': [], 'stds': []}
    bias_stats   = {'epochs': [], 'means': [], 'stds': []}
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        model.train()
        total_loss = 0.0
        batches = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)
            total_loss += svi.step(images, labels)

            batch_loss = svi.step(images, labels)

            with torch.no_grad():
                kl, nll = compute_kl_nll_full(model, guide, images, labels)

            # 3) print them side by side
            total_kl_nll = kl.item() + nll.item()
            print(f" batch_loss: {batch_loss:.2f}  |  KL+NLL: {total_kl_nll:.2f}"
                f"  (KL={kl.item():.2f}, NLL={nll.item():.2f})")
            
            batches += 1

        avg_loss = total_loss / batches
        epoch_losses.append(avg_loss)
        print(f"Epoch {epoch} - ELBO Loss: {avg_loss:.4f}")


        if epoch == 1 or epoch % 10 == 0 or epoch == num_epochs:
            model.eval(); guide.eval()
            correct, total = 0, 0
            with torch.no_grad():
                for images, labels in tqdm(train_loader, desc=f"Acc check epoch {epoch}"):
                    images, labels = images.to(device), labels.to(device)
                    trace = pyro.poutine.trace(guide).get_trace(images)
                    replayed = pyro.poutine.replay(model, trace=trace)
                    logits = replayed(images)
                    preds = torch.argmax(logits, dim=1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)

            acc = correct/total
            epoch_accuracies.append(acc); accuracy_epochs.append(epoch)
            print(f"Epoch {epoch} - Train Acc: {acc*100:.2f}%")

            # record stats...
            w_means, w_stds, b_means, b_stds = [], [], [], []
            for name, param in pyro.get_param_store().items():
                if 'loc' in name:
                    w_means.append(param.mean().item()); w_stds.append(param.std().item())
                elif 'scale' in name:
                    b_means.append(param.mean().item()); b_stds.append(param.std().item())
            weight_stats['epochs'].append(epoch)
            weight_stats['means'].append(w_means)
            weight_stats['stds'].append(w_stds)
            bias_stats['epochs'].append(epoch)
            bias_stats['means'].append(b_means)
            bias_stats['stds'].append(b_stds)

            for name, param in pyro.get_param_store().items():
                if 'loc' in name or 'scale' in name:
                    print(f"{name}: {param.detach().cpu().numpy()}")

            # only save when accuracy improves
            if acc > best_acc:
                best_acc = acc
                fname_model = model_filename_pattern.format(activation=act_name, prior=prior_name, epoch="best", timestamp=timestamp)
                fname_guide = guide_filename_pattern.format(activation=act_name, prior=prior_name, epoch="best", timestamp=timestamp)
                fname_ps    = param_store_filename_pattern.format(activation=act_name, prior=prior_name, epoch="best", timestamp=timestamp)

                torch.save(model.state_dict(), os.path.join(save_dir, fname_model))
                torch.save(guide.state_dict(), os.path.join(save_dir, fname_guide))
                pyro.get_param_store().save(os.path.join(save_dir, fname_ps))
                print(f"  ↳ Saved: {fname_model}, {fname_guide}, {fname_ps}")

    # save losses per epoch in a csv file, with consistent file naming
    accuracies_df = pd.DataFrame({
        'epoch': accuracy_epochs,
        'accuracy': epoch_accuracies
    })
    accuracies_df.to_csv(os.path.join(save_dir,accuracies_filename_pattern.format(activation=act_name, prior=prior_name, timestamp=timestamp)), index=False)

    loss_df = pd.DataFrame({
        'epoch': list(range(1, epoch + 1)),
        'loss': epoch_losses
    })
    loss_df.to_csv(os.path.join(save_dir,losses_filename_pattern.format(activation=act_name, prior=prior_name, timestamp=timestamp)), index=False)
            
    # save model configuration in a json file
    config = {
        'activation': act_name,
        'prior': prior_name,
        'num_epochs': num_epochs,
        'best_accuracy_at_epoch': accuracy_epochs[np.argmax(epoch_accuracies)],
        'best_accuracy': best_acc,
        'batch_size': train_loader.batch_size,
        'train_size': len(train_loader.dataset),
        'prior_params': {
            'mu': model.prior_mu.item(),
            'b': model.prior_b.item()
        },
    }
    config_filename = model_config_filename_pattern.format(activation=act_name, prior=prior_name, timestamp=timestamp)

    with open(os.path.join(save_dir, config_filename), 'w') as f:
        import json
        json.dump(config, f, indent=4)
        print(f"Configuration saved to {config_filename}")

    return epoch_losses, epoch_accuracies, accuracy_epochs, weight_stats, bias_stats, os.path.join(save_dir, fname_model), os.path.join(save_dir, fname_guide), os.path.join(save_dir, fname_ps), timestamp


In [9]:
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.optim import Adam
from tqdm import tqdm
import pandas as pd

In [10]:
num_classes = 10

bayesian_model = BayesianCNNSingleFC(num_classes,
        device,
        activation='relu',
        prior_dist='gaussian',
        prior_params={'mu': 0.0, 'b': 2.0})

#act_name  = getattr(bayesian_model, 'activation', getattr(bayesian_model, 'activation', 'act'))
#prior_name = getattr(bayesian_model, 'prior_dist', 'prior')

#guide = AutoDiagonalNormal(bayesian_model)

from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer.autoguide.initialization import init_to_median
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO

# 1) construct your guide so its locs start at p(w).mean=0
guide = AutoDiagonalNormal(
    bayesian_model,
    init_loc_fn=init_to_median(num_samples=1),   # all μ_q ← prior mean (0)
    init_scale=0.1               # set initial σ_q=0.1
)

optimizer = Adam({"lr": 1e-3,"weight_decay": 1e-4})  # Increased from 1e-4 to 1e-3, and added weight decay
svi = pyro.infer.SVI(model=bayesian_model,
                     guide=guide,
                     optim=optimizer,
                     #loss=pyro.infer.Trace_ELBO(num_particles=1,
                     #                           )) #TODO
                     loss=elbo)


Using prior distribution: gaussian with mu=0.0 and b=2.0


In [11]:
pyro.clear_param_store()

# Ensure model and guide are on the correct device
bayesian_model.to(device)
guide.to(device)

train_loader, test_loader = load_data(batch_size=54)

In [12]:
losses, accuracies, accuracy_epochs, weight_stats, bias_stats, best_model_path, best_guide_path, best_param_store_path, experiment_timestamp = train_svi_with_stats(
    bayesian_model,
    guide,
    svi,
    train_loader,
    device,
    num_epochs=10,
    save_epochs=None,
    save_dir='results_GP_eurosat_TEST')

Epoch 1/10:   1%|          | 4/360 [00:01<01:16,  4.68it/s]

 batch_loss: 163475.29  |  KL+NLL: 115386.82  (KL=114798.72, NLL=588.10)
 batch_loss: 163571.22  |  KL+NLL: 115480.65  (KL=114753.08, NLL=727.57)
 batch_loss: 163388.36  |  KL+NLL: 115250.76  (KL=114703.83, NLL=546.93)
 batch_loss: 162786.83  |  KL+NLL: 115260.36  (KL=114680.80, NLL=579.56)
 batch_loss: 162547.89  |  KL+NLL: 115141.22  (KL=114632.80, NLL=508.42)


Epoch 1/10:   2%|▎         | 9/360 [00:01<00:35, 10.01it/s]

 batch_loss: 162520.00  |  KL+NLL: 115112.70  (KL=114609.34, NLL=503.36)
 batch_loss: 162420.62  |  KL+NLL: 115051.90  (KL=114574.30, NLL=477.60)
 batch_loss: 162384.98  |  KL+NLL: 115198.89  (KL=114550.73, NLL=648.15)
 batch_loss: 162163.46  |  KL+NLL: 114842.22  (KL=114481.93, NLL=360.29)
 batch_loss: 161907.58  |  KL+NLL: 114784.74  (KL=114451.66, NLL=333.09)


Epoch 1/10:   4%|▍         | 14/360 [00:01<00:24, 14.25it/s]

...

 batch_loss: 400.38  |  KL+NLL: 112836.34  (KL=112689.55, NLL=146.79)
 batch_loss: 358.12  |  KL+NLL: 112924.94  (KL=112832.34, NLL=92.61)
 batch_loss: 384.12  |  KL+NLL: 113246.97  (KL=113129.37, NLL=117.60)
 batch_loss: 385.24  |  KL+NLL: 113095.90  (KL=113000.00, NLL=95.90)
 batch_loss: 372.96  |  KL+NLL: 112988.61  (KL=112878.38, NLL=110.23)


Epoch 10/10: 100%|██████████| 360/360 [00:17<00:00, 20.38it/s]


 batch_loss: 374.43  |  KL+NLL: 113159.55  (KL=113036.30, NLL=123.24)
 batch_loss: 389.71  |  KL+NLL: 113083.71  (KL=112963.16, NLL=120.54)
 batch_loss: 428.87  |  KL+NLL: 113007.77  (KL=112892.84, NLL=114.92)
 batch_loss: 401.30  |  KL+NLL: 112996.53  (KL=112861.52, NLL=135.00)
Epoch 10 - ELBO Loss: 491.7120


Acc check epoch 10: 100%|██████████| 360/360 [00:04<00:00, 79.42it/s]

Epoch 10 - Train Acc: 18.52%
AutoDiagonalNormal.loc: [ 0.00305209  0.70558995 -0.05031909 ... -1.1473572  -1.7199486
 -1.7610376 ]
AutoDiagonalNormal.scale: [1.9176738  1.8905535  1.9324698  ... 0.49652037 1.0352587  1.0843289 ]
Configuration saved to config_relu_gaussian_20250715_100559.json


In [13]:
def plot_training_results_with_stats(losses, accuracies, accuracy_epochs, weight_stats, bias_stats, act_name, prior_name, timestamp):
    """Plot training results with weight and bias statistics"""
    plt.figure(figsize=(16, 12))
    
    # Plot 1: Training Loss
    plt.subplot(2, 2, 1)
    plt.plot(range(1, len(losses) + 1), losses)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('ELBO Loss')
    plt.grid(True)
    
    # Plot 2: Training Accuracy
    plt.subplot(2, 2, 2)
    plt.plot(accuracy_epochs, accuracies, 'o-')
    plt.title('Training Accuracy (Every 10 Epochs)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    
    # Plot 3: Weight Statistics Boxplot
    plt.subplot(2, 2, 3)
    weight_data = []
    weight_labels = []
    
    for i, epoch in enumerate(weight_stats['epochs']):
        # Combine means and stds for this epoch
        epoch_data = weight_stats['means'][i] + weight_stats['stds'][i]
        weight_data.append(epoch_data)
        weight_labels.append(f'Epoch {epoch}')
    
    if weight_data:
        bp1 = plt.boxplot(weight_data, labels=weight_labels, patch_artist=True)
        for patch in bp1['boxes']:
            patch.set_facecolor('lightblue')
    
    plt.title('LOC Statistics Distribution')
    plt.xlabel('Epoch')
    plt.ylabel('LOC Values')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Plot 4: Bias Statistics Boxplot
    plt.subplot(2, 2, 4)
    bias_data = []
    bias_labels = []
    
    for i, epoch in enumerate(bias_stats['epochs']):
        # Combine means and stds for this epoch
        epoch_data = bias_stats['means'][i] + bias_stats['stds'][i]
        bias_data.append(epoch_data)
        bias_labels.append(f'Epoch {epoch}')
    
    if bias_data:
        bp2 = plt.boxplot(bias_data, tick_labels=bias_labels, patch_artist=True)
        for patch in bp2['boxes']:
            patch.set_facecolor('lightcoral')
    
    plt.title('SCALE Statistics Distribution')
    plt.xlabel('Epoch')
    plt.ylabel('SCALE Values')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'results_GP_eurosat/bayesian_cnn_training_results_{act_name}_{prior_name}_{timestamp}.png')
    plt.show()

In [14]:
act_name = bayesian_model.activation_fn.__name__ if hasattr(bayesian_model.activation_fn, '__name__') else str(bayesian_model.activation_fn)
prior_name = getattr(bayesian_model, 'prior_dist', 'prior')

In [ ]:
plot_training_results_with_stats(losses, accuracies, accuracy_epochs, weight_stats, bias_stats, act_name, prior_name, experiment_timestamp)

In [ ]:
# clear the parameter store and reload the parameter store from the best result
pyro.clear_param_store()

bayesian_model.load_state_dict(torch.load(best_model_path))
guide.load_state_dict(torch.load(best_guide_path))
pyro.get_param_store().set_state(torch.load(best_param_store_path,weights_only=False))

In [ ]:
# print confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix


def predict_data(model, loader_of_interest, num_samples=10):
    model.eval()
    guide.eval()

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in tqdm(loader_of_interest, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            logits_mc = torch.zeros(num_samples, images.size(0), model.fc1.out_features).to(device)

            for i in range(num_samples):
                guide_trace = pyro.poutine.trace(guide).get_trace(images)
                replayed_model = pyro.poutine.replay(model, trace=guide_trace)
                logits = replayed_model(images)
                logits_mc[i] = logits

            avg_logits = logits_mc.mean(dim=0)
            predictions = torch.argmax(avg_logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    return all_labels, all_predictions

In [ ]:
all_labels, all_predictions = predict_data(bayesian_model, test_loader, num_samples=10)
cm = confusion_matrix(all_labels, all_predictions)
#print accuracy from confusion matrix
accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy from confusion matrix: {accuracy * 100:.6f}%")

In [ ]:
df = pd.DataFrame({'True Label': all_labels, 'Predicted Label': all_predictions})

In [ ]:
#save the prediction label and true label in a csv file

def save_predictions_to_csv(labels, predictions, filename='predictions.csv'):
    df = pd.DataFrame({'True Label': labels, 'Predicted Label': predictions})
    df.to_csv(filename, index=False)
    print(f"Predictions saved to {filename}")

In [ ]:
save_predictions_to_csv(all_labels, all_predictions, os.path.join('results_GP_eurosat', f'predictions_{act_name}_{prior_name}_{experiment_timestamp}_{accuracy * 100:.0f}.csv'))

In [ ]:
# print pyro param store values
for name, param in pyro.get_param_store().items():
    if 'loc' in name or 'scale' in name:
        print(f"{name}: {param.detach().cpu().numpy()}")